### CNN 모델 구현하기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS = 40
BATCH_SIZE = 64

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=True,
                          download=True,
                          transform=transforms.Compose([
                                                        transforms.ToTensor(),
                                                        transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=False,
                          transform=transforms.Compose([
                                                        transforms.ToTensor(),
                                                        transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=BATCH_SIZE,
    shuffle=True
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



In [5]:
# CNN 구현
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.drop = nn.Dropout2d()  # default값 : 0.5
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x

In [6]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} \
            ({100. * batch_idx / len(train_loader):.0f}\tLoss:{loss.item():.6f}")

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print(f"[{epoch}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%")

Train Epoch: 1 [0/60000             (0	Loss:2.308054
Train Epoch: 1 [12800/60000             (21	Loss:1.111622
Train Epoch: 1 [25600/60000             (43	Loss:0.916832
Train Epoch: 1 [38400/60000             (64	Loss:0.781221
Train Epoch: 1 [51200/60000             (85	Loss:0.973091
[1] Test Loss: 0.6235, Accuracy: 76.65%
Train Epoch: 2 [0/60000             (0	Loss:0.950652
Train Epoch: 2 [12800/60000             (21	Loss:0.834859
Train Epoch: 2 [25600/60000             (43	Loss:0.778539
Train Epoch: 2 [38400/60000             (64	Loss:0.640449
Train Epoch: 2 [51200/60000             (85	Loss:0.772858
[2] Test Loss: 0.5513, Accuracy: 78.79%
Train Epoch: 3 [0/60000             (0	Loss:0.701662
Train Epoch: 3 [12800/60000             (21	Loss:0.740427
Train Epoch: 3 [25600/60000             (43	Loss:0.684093
Train Epoch: 3 [38400/60000             (64	Loss:0.497685
Train Epoch: 3 [51200/60000             (85	Loss:0.561458
[3] Test Loss: 0.4826, Accuracy: 81.89%
Train Epoch: 4 [0/60000  

### ResNet으로 컬러데이터셋 적용

In [68]:
EPOCHS = 300
BATCH_SIZE = 128

In [10]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                    train=True,
                    download=True,
                    transform=transforms.Compose([
                                                  transforms.RandomCrop(32, padding=4), ## ?????????
                                                  transforms.RandomHorizontalFlip(),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                    ])),
    batch_size=BATCH_SIZE, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                    train=False,
                    transform=transforms.Compose([
                                                  transforms.ToTensor(),
                                                  transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                    ])),
    batch_size=BATCH_SIZE, shuffle=True
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./.data/cifar-10-python.tar.gz to ./.data


In [12]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)    # image scale 유지됨
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        
        if stride != 1 or in_planes != planes:  # ??????????????
            # Sequential(): 여러 모듈(nn.Module)을 하나의 모듈로 묶는 역할
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)     # p.176에 왜 shortcut이 2번만 더해지는건지...?????????????????? 6번 더해져야 하는거아닌가?????????????? print(model)에는 표시됨
        out = F.relu(out)
        return out

In [62]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,    # 3 channel -> 16 channel 로
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1)   # (16ch -> 16ch) * 2
        self.layer2 = self._make_layer(32, 2, stride=2)   # (16ch -> 32ch) + (32ch -> 32ch)
        self.layer3 = self._make_layer(64, 2, stride=2)   # (32ch -> 64ch) + (64ch -> 64ch)
        self.linear = nn.Linear(64, num_classes)
    
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)  # 첫번째 basicblock 말고 모두 stride=1로
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)  # kernel_size=8
        out = out.view(out.size(0), -1) 
        out = self.linear(out)
        return out

In [63]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005)    # weight_decay: L2 penalty 사용
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50,  # learning rate decay (50epoch마다 learning_rate *= gamma)
                                      gamma=0.1)

In [64]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [65]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [66]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()    
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)   # 100. 을 쓰는 이유????????????????????????????????
    return test_loss, test_accuracy

In [67]:
for epoch in range(1, EPOCHS + 1):
    scheduler.step()    # 학습률을 조금 낮춰주는 단계
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print(f"[{epoch}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%")

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1] Test Loss: 1.3266, Accuracy: 54.03%
[2] Test Loss: 1.3234, Accuracy: 54.78%
[3] Test Loss: 1.3692, Accuracy: 57.46%
[4] Test Loss: 1.5892, Accuracy: 56.28%
[5] Test Loss: 1.8231, Accuracy: 50.31%
[6] Test Loss: 0.8771, Accuracy: 71.33%
[7] Test Loss: 1.0619, Accuracy: 66.47%
[8] Test Loss: 0.9842, Accuracy: 68.37%
[9] Test Loss: 0.9938, Accuracy: 67.42%
[10] Test Loss: 0.9619, Accuracy: 68.59%
[11] Test Loss: 0.8105, Accuracy: 74.39%
[12] Test Loss: 0.9646, Accuracy: 68.14%
[13] Test Loss: 1.0732, Accuracy: 68.21%
[14] Test Loss: 0.8273, Accuracy: 72.95%
[15] Test Loss: 1.0032, Accuracy: 68.83%
[16] Test Loss: 1.1059, Accuracy: 68.43%
[17] Test Loss: 0.7534, Accuracy: 74.22%
[18] Test Loss: 1.0422, Accuracy: 65.91%
[19] Test Loss: 0.9137, Accuracy: 70.39%
[20] Test Loss: 0.7133, Accuracy: 76.11%
[21] Test Loss: 0.9271, Accuracy: 70.20%
[22] Test Loss: 1.0546, Accuracy: 67.28%
[23] Test Loss: 0.8053, Accuracy: 72.92%
[24] Test Loss: 0.8795, Accuracy: 71.52%
[25] Test Loss: 0.8624, A